In [1]:
import tensorflow as tf
import numpy as np
import scipy

import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
import glob, os, sys, gc
import pickle

sys.path.insert(0,'..') # add parent directory to path
from prior_attention import build_prior_attentions
import utils
from featureextractor import FeatureExtractor
from config import Config

# this is important to make sure the Config is reloaded when modified
%load_ext autoreload
%autoreload 2
# %pprint

Using TensorFlow backend.


In [2]:
dict(vars(Config))

{'__module__': 'config',
 'main_path': '/home/mikey/Data/POETdataset/PascalImages',
 'class_names': ['dog',
  'aeroplane',
  'boat',
  'bicycle',
  'cat',
  'cow',
  'diningtable',
  'horse',
  'motorbike',
  'sofa'],
 'IMG_SIZE': 480,
 'PATCH_WIDTH': 60,
 'SMALLER_IMG_SIZE': 150,
 'T': 64,
 'new_dir': 'soft_attention_features_60',
 'new_dir_img': 'soft_attention_images_60',
 'name_to_class_dict': {'dog': 0,
  'aeroplane': 1,
  'boat': 2,
  'bicycle': 3,
  'cat': 4,
  'cow': 5,
  'diningtable': 6,
  'horse': 7,
  'motorbike': 8,
  'sofa': 9},
 'train_ids': ['horse_2010_004477.jpg',
  'cat_2010_001351.jpg',
  'horse_2008_006434.jpg',
  'dog_2010_002586.jpg',
  'cat_2008_007239.jpg',
  'cat_2010_002305.jpg',
  'cat_2009_003419.jpg',
  'sofa_2008_004670.jpg',
  'cow_2009_003189.jpg',
  'dog_2010_002808.jpg',
  'horse_2009_000409.jpg',
  'diningtable_2010_004361.jpg',
  'dog_2009_001535.jpg',
  'cat_2011_000973.jpg',
  'cow_2010_001694.jpg',
  'sofa_2011_001192.jpg',
  'cat_2009_001037.jpg

In [3]:
# extract features for a list of ids
def extract_features(name_to_class_dict, main_path, new_dir, model, img_size, patch_width, ids, mode):
    
    if os.path.exists(new_dir):
        print("Directory", new_dir, "already exists. Exitting...")
#         return
    else:
        os.makedirs(new_dir)
    
    print("Extracting features in mode", mode)
    
    try:
        for class_name, class_number in Config.name_to_class_dict.items():
            files = []
            class_ids = []
            print(class_name)
            for i, file_name in enumerate(ids):
                if class_name in file_name:
                    files.append(main_path + os.path.sep + file_name)
                    class_ids.append(file_name)

            features = FeatureExtractor.get_features(files, model, img_size, patch_width)

            utils.save_object(features, new_dir + os.path.sep + class_name + '_'+mode+'_features.pkl')
            utils.save_object(class_ids, new_dir + os.path.sep + class_name + '_'+mode+'_names.pkl')
    except:
        os.rmdir(new_dir)
    
# save resized images to fit in memory when loading
def save_images(name_to_class_dict, main_path, new_dir_img, img_size, ids, mode):
    
    if os.path.exists(new_dir_img):
        print("Directory", new_dir_img, "already exists. Exitting...")
#         return
    else:
        os.makedirs(new_dir_img)
    
    print("Saving images in mode", mode)
        
    try:
        for class_name, class_number in name_to_class_dict.items():
            files = []
            print(class_name)
            for i, file_name in enumerate(ids):
                if class_name in file_name:
                    files.append(main_path + os.path.sep + file_name)

            images = FeatureExtractor._load_images(files, img_size=img_size)
            print(images.shape)
            save_path = new_dir_img + os.path.sep + class_name + '_' + mode+'.pkl'
            print("Saving...", save_path)
            utils.save_object(images, save_path)
    except:
        os.rmdir(new_dir_img)

# This code above should check if the pickle file already exists not if the directory exists

In [4]:
model = FeatureExtractor.get_feature_extraction_model(img_size = Config.PATCH_WIDTH)

In [5]:
# load from config
feature_extractor_args = [Config.name_to_class_dict, Config.main_path, Config.new_dir, model, 
                          Config.IMG_SIZE, Config.PATCH_WIDTH]
image_saving_args = [Config.name_to_class_dict, Config.main_path, Config.new_dir_img, 
                      Config.SMALLER_IMG_SIZE]

# extract features for patches
extract_features(*feature_extractor_args, mode='train', ids=Config.train_ids)
extract_features(*feature_extractor_args, mode='test', ids=Config.test_ids)

# save smaller images for visualization
save_images(*image_saving_args, mode='train', ids=Config.train_ids)
save_images(*image_saving_args, mode='test', ids=Config.test_ids)

Directory soft_attention_features_60 already exists. Exitting...
Extracting features in mode train
dog
(1128, 8, 8, 60, 60, 3)
72192/72192 [==============================] - 41s 565us/step
(72192, 2048)
aeroplane
(599, 8, 8, 60, 60, 3)
38336/38336 [==============================] - 20s 534us/step
(38336, 2048)
boat
(454, 8, 8, 60, 60, 3)
29056/29056 [==============================] - 15s 533us/step
(29056, 2048)
bicycle
(480, 8, 8, 60, 60, 3)
30720/30720 [==============================] - 16s 535us/step
(30720, 2048)
cat
(943, 8, 8, 60, 60, 3)
60352/60352 [==============================] - 32s 535us/step
(60352, 2048)
cow
(265, 8, 8, 60, 60, 3)
16960/16960 [==============================] - 9s 536us/step
(16960, 2048)
diningtable
(427, 8, 8, 60, 60, 3)
27328/27328 [==============================] - 15s 535us/step
(27328, 2048)
horse
(428, 8, 8, 60, 60, 3)
27392/27392 [==============================] - 15s 536us/step
(27392, 2048)
motorbike
(449, 8, 8, 60, 60, 3)
28736/28736 [==========